# Lab 06 Stochastic Programming

Made & Presented by Bo Tang

In this lab, we will explore the fundamental concepts of Stochastic Programming and its application in handling uncertainty in decision-making processes. Our focus will be on two-stage stochastic programming, where decisions are made in two phases: a first-stage decision before uncertainty is revealed and a second-stage recourse decision after uncertainty is realized. We will formulate the problem in its extensive form and demonstrate the Sample Average Approximation (SAA) method.

In [ ]:
# install gurobipy first
! pip install gurobipy

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

Stochastic Programming is an optimization framework designed to handle decision-making under uncertainty. Unlike deterministic optimization, where all parameters are assumed to be known with certainty, stochastic programming explicitly incorporates randomness into the model. This allows for more realistic and robust decision-making, especially in situations where future outcomes are uncertain.

In many real-world applications—such as finance, energy systems, and supply chain management—key parameters are uncertain and must be estimated based on historical data or probabilistic models.

For example:

- **Finance**: In portfolio optimization, future stock prices and interest rates are unknown and volatile. A stochastic programming model helps investors allocate assets by considering multiple market scenarios, ensuring a balance between expected returns and risk exposure.
- **Energy Systems**: In power generation planning, the availability of renewable energy sources (such as wind speed or solar radiation) is highly uncertain. A stochastic optimization approach allows grid operators to schedule electricity generation efficiently while ensuring reliability under different weather conditions.
- **Supply Chain Management**: In inventory control and logistics, customer demand and supplier lead times are unpredictable. A company using stochastic programming can determine optimal order quantities and distribution strategies that minimize costs while maintaining service levels across various demand scenarios.

By integrating randomness into optimization models, stochastic programming helps decision-makers develop robust, effect strategies that perform well across a range of potential future outcomes.


### **1. General Stochastic Program**

The general formulation of a stochastic program is:

$$
\min_{\mathbf{x} \in \mathcal{X(\xi)}} \mathbb{E} [ f(\mathbf{x}, \xi) ]
$$

- $\mathbf{x}$: represents the decision variables, constrained to a feasible set $\mathcal{X}(\xi)$.  
- $\xi$: Random vector representing uncertain parameters, defined on some probability space $\Omega$.
- $f(\mathbf{x}, \xi)$: Objective function that quantifies the performance or cost of a decision.
- $\mathbb{E} [ f(\mathbf{x}, \xi) ] = \int_{\Omega} f(\mathbf{x}, \xi) d \mathbb{P}$: Expected value of the objective function over the uncertainty space.



### **2. Two-Stage Stochastic Linear Program**

A common structure in stochastic programming is the two-stage model. In this approach, decisions are divided into:

1. **First-stage decisions $\mathbf{x}$ (here-and-now)**: These are decisions made before the uncertainty $\xi$ is realized. They must be chosen without knowing the exact outcome of uncertain parameters.

2. **Second-stage decisions $\mathbf{y}$ (wait-and-see)**: Once the uncertainty $\xi$ is observed, additional adjustments (or recourse actions) can be made to respond to the realized scenario. These decisions depend on both the first-stage choices and the observed uncertainty.

#### 2.1 Examples of Two-Stage Stochastic Programs

- Inventory Management:

    - First-stage ($\mathbf{x}$): A retailer orders a fixed amount of products before knowing the exact customer demand.
    - Second-stage ($\mathbf{y}$): Once demand is realized, the retailer places emergency orders (if demand is high) or discounts excess stock (if demand is low).

- Hiring Staff for an Event

    - First-stage ($\mathbf{x}$): An event planner hires a certain number of staff before knowing how many attendees will show up.
    - Second-stage ($\mathbf{y}$): If attendance is higher than expected, temporary staff are hired at a premium rate; if attendance is lower, the extra staff are reassigned or released early.

#### 2.2 Mathematical Formulation

The general form for a two-stage stochastic linear program can be written as:
$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbf{c}^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
where:
- **$\mathbf{x}$**: First-stage decision variables, chosen before the uncertainty $\xi$ is revealed.
- **$\mathbf{c}^{\top} \mathbf{x}$**: Direct cost associated with first-stage decisions.
- **$\mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]$**: Expected future cost of making second-stage adjustments once uncertainty is observed.

The recourse function is:
$$
Q(\mathbf{x}, \xi) := \min_{\mathbf{y}} \mathbf{q}(\xi)^{\top} \mathbf{y}
$$
Subject to
$$
\mathbf{W}(\xi) \mathbf{y} = \mathbf{h}(\xi) - \mathbf{T}(\xi) \mathbf{x}
$$
$$
\mathbf{y} \geq \mathbf{0}
$$

$ \mathbf{q}(\xi), \mathbf{h}(\xi), \mathbf{W}(\xi), \mathbf{T}(\xi) $ are scenario-dependent parameters.

#### 2.3 Key Insights

- The first-stage problem makes decisions before knowing the exact realization of uncertainty, but aims to minimize the total expected cost.
- The second-stage problem adjusts the decision after the uncertainty is observed, ensuring feasibility and mitigating additional costs.
- The expectation $\mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]$ represents a high-dimensional integral, which is often approximated using Sample Average Approximation (SAA) or decomposition methods like Benders decomposition.

#### 2.4 Question:

**1. What if $\mathbf{c}$ is random?**
$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbb{E}_{\xi} \left[ \mathbf{c}(\xi)^{\top} \mathbf{x} + Q(\mathbf{x}, \xi) \right]
$$
$$
= \min_{\mathbf{x} \in \mathcal{X}} \mathbb{E}_{\xi} \left[ \mathbf{c}(\xi)^{\top} \mathbf{x} \right] + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
$$
= \min_{\mathbf{x} \in \mathcal{X}} \mathbb{E}_{\xi} \left[ \mathbf{c}(\xi) \right]^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
    Hence, we can just take $\mathbb{E}_{\xi} \left[ \mathbf{c}(\xi) \right]$ as $\mathbf{c}$.

**2. Why cannot we use $Q \big( \mathbf{x}, \mathbb{E}_{\xi} [ \xi) ] \big)$ directly?**

We cannot replace the expected recourse function $\mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]$ with $Q \big( \mathbf{x}, \mathbb{E}_{\xi} [ \xi ] \big)$.

This follows from **Jensen’s inequality**, which states that for a nonlinear function $f$,  
$$
\mathbb{E} [ f(\xi) ] \neq f( \mathbb{E} [\xi] )
$$
*Why are recourse functions nonlinear?*

### **3. Extensive Form**  

When the **number of scenarios is finite**, the **extensive form** explicitly includes all possible scenarios in a **single deterministic equivalent model**. This formulation allows the decision-maker to account for multiple possible realizations of uncertainty in a structured way.  

For example, consider a **two-stage stochastic problem with $ \Xi $ scenarios**, each occurring with probability $ \mathcal{p}_{\xi} $. The extensive form combines the first-stage decision variables with **recourse variables** for each scenario.  

#### **Mathematical Formulation**  

$$
\min_{\mathbf{x}, \mathbf{y}_{\xi}} \mathbf{c}^{\top} \mathbf{x} + \sum_{\xi=1}^{\Xi} \mathcal{p}_{\xi} \mathbf{q}_{\xi}^{\top} \mathbf{y}_{\xi}
$$

subject to:  

$$
\mathbf{x} \in \mathcal{X}
$$

$$
\mathbf{W}_{\xi} \mathbf{y}_{\xi} = \mathbf{h}_{\xi} - \mathbf{T}_{\xi} \mathbf{x}, \quad \forall \xi \in \Xi
$$

$$
\mathbf{y}_{\xi} \geq \mathbf{0}, \quad \forall \xi \in \Xi
$$

where:  
- **$ \mathbf{x} $**: First-stage decision variables (same across all scenarios).  
- **$ \mathbf{y}_{\xi} $**: Second-stage (recourse) decision variables for each scenario $ \xi $.  
- **$ \mathbf{c}^{\top} \mathbf{x} $**: First-stage cost.  
- **$ \mathbf{q}_{\xi}^{\top} \mathbf{y}_{\xi} $**: Scenario-dependent recourse cost under each senario $\xi$.  
- **$ \mathcal{p}_{\xi} $**: Probability of scenario $ \xi $.  
- **$ \mathbf{W}_{\xi}, \mathbf{h}_{\xi}, \mathbf{T}_{\xi} $**: Scenario-dependent constraint parameters.


### **4. Sample Average Approximation (SAA)**  

In many real-world applications, the **number of scenarios is either infinite, extremely large, or computationally intractable** ($\int_{\Omega} Q(\mathbf{x}, \xi) d \mathbb{P}$) to handle explicitly. Instead of evaluating the expectation exactly, we approximate it using Sample Average Approximation (SAA). This method replaces the true expectation with an empirical average computed from a finite set of sampled scenarios, making the problem computationally feasible.

#### 4.1 Mathematical Formulation

Given a set of $N$ independent samples (scenarios) of uncertainty, $ \{ \xi_1, \xi_2, \dots, \xi_N \} $, the expectation in the two-stage stochastic program:

$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbf{c}^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$

is approximated by the sample average:

$$
\min_{\mathbf{x}, \mathbf{y}_n} \mathbf{c}^{\top} \mathbf{x} + \frac{1}{N} \sum_{n=1}^{N} Q(\mathbf{x}, \xi_n)
$$

where the recourse functio* for each sample scenario $ \xi_n $ is:

$$
Q(\mathbf{x}, \xi_n) = \min_{\mathbf{y}_n} \mathbf{q}(\xi_n)^{\top} \mathbf{y}_n
$$

subject to:

$$
\mathbf{W}(\xi_n) \mathbf{y}_n = \mathbf{h}(\xi_n) - \mathbf{T}(\xi_n) \mathbf{x}, \quad \forall n = 1, \dots, N
$$

$$
\mathbf{y}_n \geq \mathbf{0}, \quad \forall n = 1, \dots, N
$$

where:  
- **$ N $**: Number of sampled scenarios.  
- **$ \xi_n $**: Sampled realization of the uncertainty.  
- **$ \mathbf{y}_n $**: Recourse decision corresponding to scenario $ \xi_n $.  
- **$ \frac{1}{N} \sum_{n=1}^{N} Q(\mathbf{x}, \xi_n) $**: Empirical approximation of the expected recourse cost.  

#### 4.2 Why Use SAA?
- **Computational Feasibility**: When the number of possible scenarios is infinite or too large, computing the expectation exactly is impractical. SAA provides a **tractable approximation**.  
- **Statistical Justification**: By the law of large numbers, as $ N $ increases, the SAA solution converges to the true optimal solution.  
- **Decomposition-Friendly**: Since each scenario $ \xi_n $ is treated separately, SAA can be efficiently solved using advanced decomposition method.  

#### 4.3 Choosing the Number of Samples $ N $  
- Larger $ N $ improves accuracy, but increases computational cost.  
- A practical approach is to incrementally increase $ N $ until the optimal solution stabilizes.  
- Statistical techniques like Monte Carlo confidence intervals can be used to determine an appropriate sample size.  

SAA provides a practical and efficient way to approximate stochastic programs, balancing computational efficiency with solution accuracy.


### **5. Bender's Decomposition (Extended Reading)**

**Note: Extended Reading Only:  Benders' decomposition is a more advanced technique that is not required for this course but is included here for those interested in further study. Benders’ decomposition improves efficiency by breaking large problems into smaller, more manageable subproblems.**

While the extensive form captures the entire problem, it can quickly become computationally infeasible for problems with many scenarios due to the large number of variables and constraints. This is where decomposition methods come in.

$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbf{c}^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
can be converted as
$$
\min_{\mathbf{x} \in \mathcal{X}, \eta} \mathbf{c}^{\top} \mathbf{x} + \sum_{\xi}^{\Xi} \mathcal{p}_{\xi} \eta_{\xi}
$$
Subject to
$$
\eta_{\xi} \geq Q_{\xi}(\mathbf{x})
$$

For $Q_{\xi}(\mathbf{x})$, the primal form is
$$
\min_{\mathbf{y}} \mathbf{q}_{\xi}^{\top} \mathbf{y}
$$
Subject to
$$
\mathbf{W}_{\xi} \mathbf{y} = \mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}
$$
$$
\mathbf{y} \geq \mathbf{0}
$$
Therefore, we can obtain the duality:
$$
\max_{\boldsymbol{\pi}} \boldsymbol{\pi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x})
$$
Subject to
$$
\mathbf{W}_{\xi}^\top \boldsymbol{\pi} \leq \mathbf{q}_{\xi}
$$

Let define $\Pi_{\xi} = \{\boldsymbol{\pi}: \mathbf{W}_{\xi}^\top \boldsymbol{\pi} \leq \mathbf{q}_{\xi} \}$ and $V(\Pi_{\xi})$ is the finite set of extreme points of $\Pi_{\xi}$.

We can observe:
$$
\eta_{\xi} \geq Q_{\xi}(\mathbf{x})
$$
$$
\Leftrightarrow \eta_{\xi} \geq \max_{\boldsymbol{\pi}} \{ \boldsymbol{\pi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}): \boldsymbol{\pi} \in \Pi_{\xi} \}
$$
$$
\Leftrightarrow \eta_{\xi} \geq \boldsymbol{\pi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}), \quad \forall \boldsymbol{\pi} \in V(\Pi_{\xi})
$$

Therefore, we have our **master problem** for Bender's:
$$
\min_{\mathbf{x}, \mathbf{y}} \mathbf{c}^{\top} \mathbf{x} + \sum_{\xi=1}^{\Xi} \mathcal{p}_{\xi} \eta_{\xi}
$$
Subject to
$$
\mathbf{x} \in \mathcal{X}
$$
$$
\mathbf{W}_{\xi} \mathbf{y}_{\xi} = \mathbf{h}_{\xi} - \mathbf{T}_{\xi} \mathbf{x}  \quad \forall {\xi} \in {\Xi}
$$
$$
\eta_{\xi} \geq \boldsymbol{\pi}_{\xi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}) \quad \forall \boldsymbol{\pi}_{\xi} \in V_{\xi}^t, \forall \xi \in \Xi
$$
where $V_{\xi}^t$ a (small) subset of $\Pi_{\xi}$ at iteraion $t$.

Let $\mathbf{x}^t, \eta_{\xi}^t$ be an optimal solution to master the problem at iteration $t$.
1. For each $\xi$, we need to solve subproblem $Q(\mathbf{x}_t, \xi)$ to obtain a $\boldsymbol{\pi}_{\xi}^t$ with the given $\mathbf{x}_t$.
2. If $\eta_{\xi} \geq Q(\mathbf{x}_t, \xi)$, all the constraints for scenario $\xi$ are satisfied. Do nothing.
3. If $\eta_{\xi} < Q(\mathbf{x}_t, \xi)$, add the constraint $\eta_{\xi} \geq {\boldsymbol{\pi}^t_{\xi}}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x})$ to master problem.

### **6. Quick Tutotial: Gurobi Indexing**

When working with **multi-index variables** in Gurobi, it's essential to use the correct indexing style to avoid common mistakes. Below is an example illustrating proper indexing in Gurobi.

Imagine we need to assign workers to different **tasks** in multiple **shifts**.  

- **Workers**: Alice, Bob, Charlie  
- **Tasks**: Packing, Assembling  
- **Shifts**: Morning, Afternoon  

Since each assignment depends on **three indices** (worker, task, shift), we should define the decision variables accordingly.

#### 6.1 Creates a dictionary of variables

```python
workers = ["Alice", "Bob", "Charlie"]
tasks = ["Packing", "Assembling"]
shifts =  ["Morning", "Afternoon"]
assign = m.addVars(workers, tasks, shifts, vtype=GRB.BINARY, name="assign")
```

To access a specific assignment variable:
```python
assign["Alice", "Packing", "Morning"]
```

#### 6.2 defining constraints over multiple indices
```python
m.addConstrs(
    (gp.quicksum(assign[w, t, s] for t in tasks) <= 1
     for w in workers for s in shifts),
    name="MaxOneTaskPerShift")
```

This constraint ensures that each worker is assigned at most one task per shift. Let's break it down step by step:


```python
for w in workers for s in shifts
```

- This iterates over all workers (w) and all shifts (s).
- Each worker in each shift gets its own constraint.

```python
gp.quicksum(assign[w, t, s] for t in tasks)
```

- Gurobi provides `gp.quicksum()` for summing variables efficiently.
- It ensures that a worker is assigned to at most one task per shift.

### **7. Example from Lecture Note**

You own 500 acres of land in which to plant wheat (w), corn (c), and beets (b) with planting costs of 150 \$/acre for wheat, 230 \$/acre for corn, and 260 \$/acre for beets.

You require 200 T of wheat and 240 T of corn to feed your cattle.
The yield of each crop is uncertain, with expected values of 2.5, 3, and 20 T/acre for wheat corn and beets, respectively. Purchasing cost of wheat is 238 \$/T and 210 \$/T for corn. Any excess wheat can be sold for 170 \$/T, corn for 150 \$/ton, beets for 36 \$/ton (up to 6000 T), beets for 10 \$/T (beyond 6000 T).

Suppose that:
- There is a 1/3 probability that crops will be as expected
- There is a 1/3 probability that ALL crops will be 20% higher than expected
- There is a 1/3 probability that ALL crops will be 20% lower than expected.

In [ ]:
# data setup
planting_cost = {"wheat": 150, "corn": 230, "beets": 260}
purchase_cost = {"wheat": 238, "corn": 210}
selling_price = {"wheat": 170, "corn": 150, "beets_high": 36, "beets_low": 10}
feed_requirements = {"wheat": 200, "corn": 240}
land_available = 500
beet_sale_limit = 6000

In [ ]:
# scenarios and probabilities
expected_yields = {"wheat": 2.5, "corn": 3, "beets": 20}
scenarios = [1, 2, 3]
yields = {
    1: {crop: yield_val for crop, yield_val in expected_yields.items()},    # Scenario 1: Expected yields
    2: {crop: 1.2 * yield_val for crop, yield_val in expected_yields.items()},  # Scenario 2: 20% higher yields
    3: {crop: 0.8 * yield_val for crop, yield_val in expected_yields.items()}   # Scenario 3: 20% lower yields
}
probabilities = {1: 1/3, 2: 1/3, 3: 1/3}

#### 7.1 Using Expected Yields

To understand the **incremental value of Stochastic Programming (SP)**, we first solve the problem **without SP**  using a commonsense **deterministic approach**. This allows us to develop a simpler **Linear Program (LP)** based on expected yields and serves as a baseline for comparison.

Simpler approach
- Formulate the LP assuming expected yields.
- Solve the LP to determine the acreages.
- Evaluate the profit for each scenario.
- Calculate the expected profit.

First, let solve the LP with expected yields.

In [ ]:
# create Gurobi model
m = gp.Model("Simple Approach")

# decision variables
planting = m.addVars(planting_cost.keys(), vtype=GRB.CONTINUOUS, name="plants")
sales = m.addVars(selling_price.keys(), vtype=GRB.CONTINUOUS, name="sales")
purchase = m.addVars(purchase_cost.keys(), vtype=GRB.CONTINUOUS, name="purchases")

# objective function
revenue = gp.quicksum(sales[crop] * selling_price[crop] for crop in selling_price)
planting_cost_total = gp.quicksum(planting[crop] * planting_cost[crop] for crop in planting_cost)
purchase_cost_total = gp.quicksum(purchase[crop] * purchase_cost[crop] for crop in purchase_cost)
m.setObjective(revenue - planting_cost_total - purchase_cost_total, GRB.MAXIMIZE)

# constraints
m.addConstr(expected_yields["wheat"] * planting["wheat"] + purchase["wheat"] - sales["wheat"] >= feed_requirements["wheat"], "Wheat Balance")
m.addConstr(expected_yields["corn"] * planting["corn"] + purchase["corn"] - sales["corn"] >= feed_requirements["corn"], "Corn Balance")
m.addConstr(expected_yields["beets"] * planting["beets"] - sales["beets_high"] - sales["beets_low"] >= 0, "Beet Balance")
m.addConstr(sales["beets_high"] <= beet_sale_limit, "Beets Limit")
m.addConstr(planting.sum() <= land_available, "Land Balance")

# solve the model
m.optimize()

# display the results
print(f"\nNet profit: {m.objVal:.0f}")
for crop in planting:
    print(f"  plant {planting[crop].x:.0f} acres of {crop}.")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 9 columns and 13 nonzeros
Model fingerprint: 0x0c7f5310
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.02s
Presolved: 3 rows, 7 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1200000e+33   2.000000e+30   5.120000e+03      0s
       4    1.1860000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.186000000e+05

Net profit: 118600
  plant 120 acres of wheat.
  plant 80 acr

Then, we evaluate the profit for each scenario and calculate the average.

In [ ]:
def resouceFunction(scenario, planting):
    # create Gurobi model
    m = gp.Model("Resource Function")
    # decision variables
    sales = m.addVars(selling_price.keys(), vtype=GRB.CONTINUOUS, name="sales")
    purchase = m.addVars(purchase_cost.keys(), vtype=GRB.CONTINUOUS, name="purchases")
    # objective function
    revenue = gp.quicksum(sales[crop] * selling_price[crop] for crop in selling_price)
    planting_cost_total = gp.quicksum(planting[crop].x * planting_cost[crop] for crop in planting_cost)
    purchase_cost_total = gp.quicksum(purchase[crop] * purchase_cost[crop] for crop in purchase_cost)
    m.setObjective(revenue - planting_cost_total - purchase_cost_total, GRB.MAXIMIZE)
    # constraints
    m.addConstr(yields[scenario]["wheat"] * planting["wheat"].x + purchase["wheat"] - sales["wheat"] >= feed_requirements["wheat"], "Wheat Balance")
    m.addConstr(yields[scenario]["corn"] * planting["corn"].x + purchase["corn"] - sales["corn"] >= feed_requirements["corn"], "Corn Balance")
    m.addConstr(yields[scenario]["beets"] * planting["beets"].x - sales["beets_high"] - sales["beets_low"] >= 0, "Beet Balance")
    m.addConstr(sales["beets_high"] <= beet_sale_limit, "Beets Limit")
    return m

# solve for each senaraio
# scenario 1
q1 = resouceFunction(1, planting)
q1.optimize()
obj1 = q1.objVal
# scenario 2
q2 = resouceFunction(2, planting)
q2.optimize()
obj2 = q2.objVal
# scenario 3
q3 = resouceFunction(3, planting)
q3.optimize()
obj3 = q3.objVal

# result
print(f"\nProfit for Scenario 1: {obj1:.0f}")
print(f"Profit for Scenario 2: {obj2:.0f}")
print(f"Profit for Scenario 3: {obj3:.0f}")
expected_profit = probabilities[1] * obj1 + probabilities[2] * obj2 + probabilities[3] * obj3
print(f"Expected Profit: {expected_profit:.0f}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 4 rows, 6 columns and 7 nonzeros
Model fingerprint: 0x12edb664
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 6e+03]
Presolve removed 4 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1860000e+05   0.000000e+00   1.000000e+01      0s
Extra simplex iterations after uncrush: 1
       1    1.1860000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.186000000e+05
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 

**Important Note: This method optimizes $f(\mathbb{E}[\xi])$ rather than $\mathbb{E}[f(\xi)]$, meaning it is not Stochastic Programming. As a result, the solution tends to be worse.**

#### 7.2 Extensive Form

In each scenario, we can either buy crops to meet the feed requirement or sell excess production. We'll formulate an extensive form to account for these scenarios.

Question:

- Which decision variables are first-stage? Which one should be decided now and here?
- Which decision variables are second-stage? Which one can be decided in the future?

In [ ]:
# create Gurobi model
m = gp.Model("Extensive Form")

# TODO: decision variables
planting =
purchase =
sales =

# TODO: objective function
m.setObjective

# TODO: constraints
m.addConstrs

# solve the model
m.optimize()

# display the results
print(f"\nNet profit: {m.objVal:.0f}")
for crop in planting:
    print(f"  plant {planting[crop].x:.0f} acres of {crop}.")

Question:

- How much of an improvement is stochastic programming over LP with expectations?

#### 7.3 Bender's Decomposition

To implement Bender’s Decomposition based on the explanation you've provided, we can break down the process into two main components: the master problem and the subproblem for each scenario.

When we have a maximization problem, we can turn it into a minimization problem by changing the sign of the objective function. Instead of maximizing $f(x)$, we minimize $−f(x)$.

##### Master Problem

We will create the master problem in Gurobi, and for each iteration, solve the subproblems to add new constraints.

In [ ]:
# master problem creation
master = gp.Model("Benders Master")

# turn off log
master.Params.outputFlag = 0

# TODO: first-stage decision variables


# recourse approximation variables
η = master.addVars(scenarios, vtype=GRB.CONTINUOUS, lb=-1e9, name="η")

# TODO: objective function for the master problem
master.setObjective

# TODO: constraint
master.addConstr

##### Subproblems

For each scenario, we solve a subproblem to determine the second-stage decisions and obtain the dual variables. If necessary, generate a new cut and add it to the master problem.

we typically solve the primal form of the subproblem to make things more intuitive and practical. Whether we solve the primal or the dual subproblem, we can always extract the dual variables.


In [ ]:
# subproblem
def subproblem(scenario, planting):
    # create Gurobi model
    subprob = gp.Model("Subproblem")
    # turn off log
    subprob.Params.outputFlag = 0
    # TODO: decision variables

    # TODO: objective function
    subprob.setObjective
    # TODO: constraints
    subprob.addConstr
    return subprob

##### Iterative Approach

The process iteratively adds new Bender’s cuts to the master problem, updating it with the information from the subproblems until convergence is achieved.

In [ ]:
# init cnt
iter = 0
# Bender's decomposition loop
while True:
    # count
    iter += 1
    # initialize a flag to check if any cuts were added
    cuts_added = False
    # solve the master
    master.optimize()
    print(f"Iteration {iter}, Objective Value: {-master.objVal:.0F}")
    for crop in planting:
        print(f"  plant {planting[crop].x:.0f} acres of {crop}.")
    print("\n")
    # for each scenario
    for sc in scenarios:
        # solve subproblem for the current scenario
        subprob = subproblem(sc, planting)
        subprob.optimize()
        # get the objective value of the subproblem
        Q_value = subprob.objVal
        # check adding a cut
        if η[sc].x < Q_value - 1e-6:
            # extract dual variables from subproblem constraints
            dual_wheat = subprob.getConstrByName("Wheat Balance").pi
            dual_corn = subprob.getConstrByName("Corn Balance").pi
            dual_beets = subprob.getConstrByName("Beet Balance").pi
            dual_beets_limit = subprob.getConstrByName("Beets Limit").pi
            # TODO: formulate the cut using dual values and add it to the master problem
            cut_expr =
            master.addConstr(η[sc] >= cut_expr, f"BendersCut {sc} at Iter {iter}")
            cuts_added = True
    # if no new cuts are added, stop the loop
    if not cuts_added:
        print("Convergence reached. No more cuts needed.")
        break

In [ ]:
# display the results
print(f"\nNet profit: {-master.objVal:.0f}")
for crop in planting:
    print(f"  plant {planting[crop].x:.0f} acres of {crop}.")

Question:

- Why do we set the lower bound of $\eta$ to $-10^9$?